In [1]:
import cv2
import pprint
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import time
%matplotlib inline

In [2]:
image = cv2.imread("apple.jpg",  cv2.IMREAD_COLOR)
new_img = np.ones_like((image),dtype=np.uint8)*255
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 125, 1)
# 根据最上面的点的像素来判断后面 二值化的结果
if gray[1][1] == 0:    
    ret, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
if gray[1][1] == 255:
    ret, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    
outimg, contours, hireachy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
top_lines = []
botton_lines = []
left_lines = []
right_lines = []

for i, contour in enumerate(contours):
    area = cv2.contourArea(contour)
 
    if area > 10000:
        print(area)
        x, y, w, h  = cv2.boundingRect(contour)
        mm = cv2.moments(contour)
        
        epsilon = 0.01*cv2.arcLength(contour,True)
        approx = cv2.approxPolyDP(contour,epsilon,True)
        hull = cv2.convexHull(approx, True)
        cv2.drawContours(image, hull, -1, (0,0,0),10)
        
lines = hull[:,0,:]
dd = int((lines.max()-lines.min())/2)
point_lines = lines-(dd,dd)
for i in range(len(point_lines)):
    if point_lines[i][0] < 0 and point_lines[i][1] < 0:
        A = lines[i]
    elif point_lines[i][0] > 0 and point_lines[i][1] < 0:
        B = lines[i]
    elif point_lines[i][0] < 0 and point_lines[i][1] > 0:
        C = lines[i]
    elif point_lines[i][0] > 0 and point_lines[i][1] > 0:
        D = lines[i]
        
corners = np.float32([A,B,C,D])
canvas = np.float32([[0, 0],[600, 0],[0,400],[600,400]])
M = cv2.getPerspectiveTransform(corners, canvas)
result = cv2.warpPerspective(image, M, (600, 400))
cv2.imwrite(f"angle_done{i}.jpg",result) 

109358.0


True